In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv


In [3]:
pip install peft==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 7.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade transformers datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
import re
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
dataPath = '../input/hate-speech-and-offensive-language-dataset/labeled_data.csv'

df = pd.read_csv(dataPath)

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"&[a-z]+;", "", text)  # Remove HTML entities
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
    return text.lower()

class_mapping = {
    0: "hate speech",
    1: "offensive",
    2: "neutral"
}

df["cleanedTweet"] = df["tweet"].apply(clean_text)
df["input_text"] = "classify: " + df["cleanedTweet"]
df["output_text"] = df["class"].map(class_mapping)

df = df.dropna(subset=["output_text"])

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
lora_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  
    lora_dropout=0.1,
    bias="none"
)

In [10]:
peft_model = get_peft_model(model, lora_config)

#Tokenize data
def tokenize_data(row):
    inputs = tokenizer(
        row["input_text"],
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    labels = tokenizer(
        row["output_text"],
        max_length=32,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100  # Mask padding tokens for loss
    return {
        "input_ids": inputs["input_ids"].squeeze(0),
        "attention_mask": inputs["attention_mask"].squeeze(0),
        "labels": labels.squeeze(0),
    }


In [11]:
# Apply tokenization 
train_tokenized = train_df.apply(tokenize_data, axis=1)
test_tokenized = test_df.apply(tokenize_data, axis=1)

# ist of dictionaries
train_tokenized = train_tokenized.tolist()
test_tokenized = test_tokenized.tolist()

In [12]:
#custom dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [13]:
#datasets and dataloaders
train_dataset = CustomDataset(train_tokenized)
test_dataset = CustomDataset(test_tokenized)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

#Device setup for single GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
peft_model.to(device)


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [14]:
optimizer = AdamW(filter(lambda p: p.requires_grad, peft_model.parameters()), lr=5e-5)

#Training Loop
epochs = 6
peft_model.train()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = peft_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss.mean()  # Ensure scalar loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Average Loss after epoch {epoch + 1}: {average_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/6


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Average Loss after epoch 1: 0.7850
Epoch 2/6
Average Loss after epoch 2: 0.2449
Epoch 3/6
Average Loss after epoch 3: 0.1964
Epoch 4/6
Average Loss after epoch 4: 0.1754
Epoch 5/6
Average Loss after epoch 5: 0.1635
Epoch 6/6
Average Loss after epoch 6: 0.1570


In [16]:
peft_model.eval()
all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Generate predictions
        generated_ids = peft_model.generate(input_ids=input_ids, attention_mask=attention_mask)
        preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # Filter out -100 from labels before decoding
        labels[labels == -100] = tokenizer.pad_token_id
        true_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        all_predictions.extend(preds)
        all_true_labels.extend(true_labels)


In [17]:
print("Classification Report:")
print(classification_report(all_true_labels, all_predictions))

output_dir = "/kaggle/working/flan_t5_peft_model"
peft_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

Classification Report:
              precision    recall  f1-score   support

 hate speech       0.47      0.23      0.31       290
     neutral       0.81      0.92      0.86       835
   offensive       0.94      0.95      0.94      3832

    accuracy                           0.90      4957
   macro avg       0.74      0.70      0.70      4957
weighted avg       0.89      0.90      0.89      4957

Model and tokenizer saved to /kaggle/working/flan_t5_peft_model
